# Visualise the ``Data`` used in this project

In [ ]:
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import xarray as xr

from sdm_eurec4a import RepositoryPath, conversions
from sdm_eurec4a.input_processing import transfer

from sdm_eurec4a.reductions import shape_dim_as_dataarray
from sdm_eurec4a.visulization import (
    adjust_lightness_array,
    set_custom_rcParams,
)

from importlib import reload

In [ ]:
def set_xticks_time(ax):
    xticks = [0, 500, 1000]
    ax.set_xticks(xticks)


def set_yticks_height(ax):
    yticks = [0, 500, 1000, 1500, 2000]
    ax.set_yticks(yticks)


def set_yticks_height_km(ax):
    yticks = [0, 0.5, 1, 1.5, 2]
    ax.set_yticks(yticks)


def set_logxticks_meter(ax):
    xticks = [1e-6, 1e-3]
    xticklabels = [r"$10^{-6}$", r"$10^{-3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logxticks_micrometer(ax):
    xticks = [1e-3, 1e0, 1e3]
    xticklabels = [r"$10^{-3}$", r"$10^{0}$", r"$10^{3}$"]
    ax.set_xticks(xticks, xticklabels)


def set_logtyticks_psd(ax):
    yticks = [1e0, 1e6]
    yticklabels = [r"$10^0$", r"$10^6$"]
    ax.set_yticks(yticks, yticklabels)


def set_yticks_lwc(ax):
    ax.set_yticks([0, 0.1, 0.2])

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
plt.rcParams.update(
    {
        "axes.spines.top": False,
        "axes.spines.right": False,
        "axes.spines.left": False,
        "axes.spines.bottom": False,
    }
)

dark_colors = adjust_lightness_array(default_colors, amount=0.5)

repo_path = RepositoryPath("levante")()
print(repo_path)

# data_subdir = "output_v3.2/stationary_no_physics"
# data_subdir = "output_v3.2/stationary_condensation"
# data_subdir = "output_v3.3/stationary_condensation"
# data_subdir = "output_v3.4/stationary_condensation"
# data_subdir = "output_v3.2/stationary_collision_condensation"
# data_subdir = "output_v3.2/stationary_collision_condensation"
data_subdir = "output_v3.4/stationary_collision_condensation"

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = Path("/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation/results/")
print(script_dir)


fig_dir = repo_path / "results" / script_dir.relative_to(repo_path) / data_subdir
print(fig_dir)

fig_dir.mkdir(parents=True, exist_ok=True)

/home/m/m301096/repositories/sdm-eurec4a
/home/m/m301096/repositories/sdm-eurec4a/notebooks/presentation/results
/home/m/m301096/repositories/sdm-eurec4a/results/notebooks/presentation/results/output_v3.4/stationary_collision_condensation


In [ ]:
identified_clouds_path = repo_path / Path(
    "data/observation/cloud_composite/processed/identified_clouds/identified_clusters_rain_mask_5.nc"
)
cloud_composite_path = repo_path / Path(
    f"data/observation/cloud_composite/processed/cloud_composite_si_units.nc"
)
drop_sondes_path = repo_path / Path("data/observation/dropsonde/processed/drop_sondes.nc")
distance_path = repo_path / Path(
    "data/observation/combined/distance_relations/distance_dropsondes_identified_clusters_rain_mask_5.nc"
)
satellite_path = repo_path / Path(
    "data/observation/satellite/samples/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20200261630156_BARBADOS-2KM-FD.level2.nc"
)

combined_path = (
    Path("/home/m/m301096/CLEO/data/") / data_subdir / "combined/eulerian_dataset_combined.nc"
)

## Load datasets

In [ ]:
# %%

distance_IC_DS = xr.open_dataset(distance_path)

cloud_composite = xr.open_dataset(cloud_composite_path)

drop_sondes = xr.open_dataset(drop_sondes_path)

drop_sondes = drop_sondes.where(drop_sondes.alt <= 1600, drop=True)

# add relative humidity to the dataset

drop_sondes["relative_humidity"] = conversions.relative_humidity_from_tps(
    temperature=drop_sondes["air_temperature"],
    pressure=drop_sondes["pressure"],
    specific_humidity=drop_sondes["specific_humidity"],
)


# Add radii and altitude arrays to datasets

cloud_composite["radius_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="radius"
)
cloud_composite["time_array"] = shape_dim_as_dataarray(
    da=cloud_composite["particle_size_distribution"], output_dim="time"
)
drop_sondes["alt_array"] = shape_dim_as_dataarray(da=drop_sondes["air_temperature"], output_dim="alt")

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

identified_clouds = xr.open_dataset(identified_clouds_path)
# select only clouds which are
# below 1500m and have a duration of at least 10s.
identified_clouds = identified_clouds.where(
    (identified_clouds["alt"] <= 1200)
    & (identified_clouds["alt"] >= 500)
    & (identified_clouds["vertical_extent"] <= 150)
    & (identified_clouds["duration"] >= np.timedelta64(3, "s"))
    # & (identified_clouds["liquid_water_content"] / identified_clouds["duration"].dt.seconds >=0.1)
    ,
    drop=True,
)

combined_output = xr.open_dataset(combined_path)

rename dataarray
rename dataarray
rename dataarray


## Calculations

### Calculate masks and coord3 and volume

In [ ]:
combined_output["sub_cloud_layer_mask"] = combined_output["gridbox"] < combined_output["max_gridbox"]
combined_output["sub_cloud_layer_mask"].attrs.update(
    long_name="Sub Cloud Layer Mask",
    description="Boolean mask indicating if the gridbox is part of the sub cloud layer",
    units="1",
)

combined_output["sub_cloud_layer_no_bottom"] = (combined_output["sub_cloud_layer_mask"]) & (
    combined_output["gridbox"] > 0
)
combined_output["sub_cloud_layer_no_bottom"].attrs.update(
    long_name="Sub Cloud Layer Mask without bottom",
    description="Boolean mask indicating if the gridbox is part of the sub cloud layer and not the bottom gridbox",
    units="1",
)


combined_output["pseudo_coord3"] = combined_output["gridbox"] * 20
combined_output["pseudo_coord3"].attrs.update(
    long_name="pseudo Coordinate 3",
    description="pseudo Coordinate 3. Which is the gridbox number times 20m. It thus is wrong for the top gridbox for each cloud. This can be identified by the variable 'max_gridbox'. So use the mask 'sub_cloud_layer_mask' to identify the sub cloud layer.",
    units="m",
)

combined_output["pseudo_coord3_normalized"] = (
    combined_output["pseudo_coord3"]
    / combined_output["pseudo_coord3"][combined_output["max_gridbox"] - 1]
)

Z = np.nan_to_num(
    x=(combined_output["pseudo_coord3"].where(combined_output["sub_cloud_layer_mask"])),
    nan=0,
)
Z_max = np.nan_to_num(
    x=(
        combined_output["pseudo_coord3"].where(
            combined_output["gridbox"] == combined_output["max_gridbox"]
        )
    )
    * 0
    + combined_output["max_gridbox"] * 20
    + (100 - 20) / 2,
    nan=0,
)
Z = Z + Z_max
combined_output["pseudo_coord3_full"] = (("gridbox", "cloud_id"), Z)
combined_output["pseudo_coord3_full"] = combined_output["pseudo_coord3_full"].where(
    combined_output["gridbox"] <= combined_output["max_gridbox"]
)
combined_output["pseudo_coord3_full"].attrs.update(
    long_name="pseudo Coordinate 3 Full",
    description="pseudo Coordinate 3. Each gridbox below cloud layer is 20m thick and cloud layer is 100m thick.",
    units="m",
)


combined_output["pseudo_gribox_volume"] = combined_output["pseudo_coord3"] * 0 + 20**3
combined_output["pseudo_gribox_volume"].attrs.update(
    long_name="Simple Pseudo Gridbox Volume",
    description="pseudo Gridbox Volume. Which is the gridbox volume for each gridbox in the sub cloud layer. It is 20m x 20m x 20m for each gridbox.",
    units="m^3",
)

combined_output["pseudo_gribox_volume_full"] = (
    ("gridbox", "cloud_id"),
    np.nan_to_num(
        x=(
            combined_output["pseudo_gribox_volume"].where(
                combined_output["gridbox"] != combined_output["max_gridbox"]
            )
        ),
        nan=20 * 20 * 100,
    ),
)
combined_output["pseudo_gribox_volume_full"] = combined_output["pseudo_gribox_volume_full"].where(
    combined_output["gridbox"] <= combined_output["max_gridbox"]
)
combined_output["pseudo_gribox_volume"].attrs.update(
    long_name="pseudo Gridbox Volume",
    description="pseudo Gridbox Volume. Which is the gridbox volume for each gridbox in the sub cloud layer. It is 20m x 20m x 20m for each gridbox in subcloud layer. 20m x 20m x 100m in cloud layer. nan for gridboxes above cloud layer.",
    units="m^3",
)

### Calculate vertical profiles and total numbers

In [ ]:
vertical_profile_mass = (
    combined_output["mass_represented"].sel(time=slice(400, None)).sum("radius_bins").mean("time")
)
vertical_profile_mass = vertical_profile_mass.where(combined_output["sub_cloud_layer_mask"])
vertical_profile_mass = 1e3 * vertical_profile_mass / combined_output["pseudo_gribox_volume"]
vertical_profile_mass.attrs.update(
    long_name="Mass",
    description="Vertical Profile of Mass. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
    units=r"$g m^{-3}$",
)
combined_output["vertical_profile_mass"] = vertical_profile_mass


combined_output["top_subcloud_mass"] = (
    combined_output["vertical_profile_mass"]
    .where(combined_output["gridbox"] == combined_output["max_gridbox"] - 1)
    .mean("gridbox")
)
combined_output["top_subcloud_mass"].attrs.update(
    long_name="Top Subcloud Mass",
    description="Mass of the top subcloud layer. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
    units=r"$g m^{-3}$",
)

combined_output["bottom_subcloud_mass"] = combined_output["vertical_profile_mass"].sel(gridbox=0)
combined_output["bottom_subcloud_mass"].attrs.update(
    long_name="Bottom Subcloud Mass",
    description="Mass of the bottom subcloud layer. The mass is averaged over time and radius bins and then divided by the gridbox volume.",
    units=r"$g m^{-3}$",
)

In [ ]:
vertical_profile_mass_fraction = (
    100
    * combined_output["vertical_profile_mass"]
    / combined_output["vertical_profile_mass"].sel(gridbox=combined_output["max_gridbox"] - 1)
)
vertical_profile_mass_fraction.attrs.update(
    long_name="Mass Fraction",
    description="Vertical Profile of Mass Fraction. The mass is divided by the mass of the topmost sub-cloud-layer gridbox.",
    units="%",
)
combined_output["vertical_profile_mass_fraction"] = vertical_profile_mass_fraction

vertical_profile_mass_diff = (
    combined_output["mass_difference"].sel(time=slice(400, None)).sum("radius_bins").mean("time")
)
vertical_profile_mass_diff = vertical_profile_mass_diff.where(
    combined_output["sub_cloud_layer_no_bottom"]
)
vertical_profile_mass_diff = 1e3 * vertical_profile_mass_diff / combined_output["pseudo_gribox_volume"]
vertical_profile_mass_diff.attrs.update(
    long_name="Mass Difference",
    description="Vertical Profile of Mass Difference. The mass difference is averaged over time and radius bins and then divided by the gridbox volume. ",
    units=r"$g m^{-3} s^{-1}$",
)
combined_output["vertical_profile_mass_diff"] = vertical_profile_mass_diff

vertical_evaporation_fraction = 100 * vertical_profile_mass_diff / vertical_profile_mass
vertical_evaporation_fraction.attrs.update(
    long_name="Vertical Evaporation Fraction",
    description="Vertical Evaporation Fraction. The mass difference is divided by the mass to get the fraction of mass which evaporates in the sub cloud layer.",
    units=r"$\%$",
)
combined_output["vertical_evaporation_fraction"] = vertical_evaporation_fraction


# total values
combined_output["mass_difference_total"] = (
    combined_output["vertical_profile_mass_diff"] * combined_output["pseudo_gribox_volume"]
).sum("gridbox", keep_attrs=True) / (
    combined_output["pseudo_gribox_volume"]
    .where(combined_output["sub_cloud_layer_no_bottom"])
    .sum("gridbox")
)
combined_output["mass_difference_total"].attrs.update(
    long_name="Total Mass Difference",
    description="Total Mass Difference. The mass difference is summed over all gridboxes and divided by the total volume of the sub cloud layer.",
    units=r"$g m^{-3} s^{-1}$",
)

combined_output["mass_total"] = combined_output["vertical_profile_mass"].sum("gridbox")
combined_output["mass_total"].attrs.update(
    long_name="Total Mass",
    description="Total Mass. The mass is summed over all gridboxes of the sub cloud layer.",
    units=r"$g$",
)

combined_output["mass_fraction_total"] = vertical_profile_mass_fraction.sel(gridbox=1)
combined_output["mass_fraction_total"].attrs.update(
    long_name="Total Mass Fraction",
    description="Total Mass Fraction. The mass fraction of the mass that reaches the bottom.",
    units="%",
)


combined_output["liquid_water_content"] = 1e3 * (
    combined_output["mass_represented"] / combined_output["pseudo_gribox_volume_full"]
)
combined_output["liquid_water_content"].attrs.update(
    long_name="Liquid Water Content",
    description="Liquid Water Content per gridbox for the sub cloud layer.",
    units=r"$g m^{-3}$",
)

combined_output["liquid_water_content_smooth"] = (
    combined_output["liquid_water_content"].rolling(time=50, center=True).mean()
)
combined_output["liquid_water_content_smooth"].attrs.update(
    long_name="Liquid Water Content Smooth",
    description="Smoothed Liquid Water Content per gridbox for the sub cloud layer.",
    units=r"$g m^{-3}$",
)


vertical_profile_lwc = (
    combined_output["liquid_water_content"]
    .sel(time=slice(400, None))
    .sum("radius_bins", keep_attrs=True)
    .mean("time", keep_attrs=True)
)
vertical_profile_lwc = vertical_profile_lwc.where(
    combined_output["gridbox"] <= combined_output["max_gridbox"]
)
vertical_profile_lwc.attrs.update(
    long_name="Liquid Water Content",
    description="Vertical Profile of liquid water content.",
)
combined_output["vertical_profile_lwc"] = vertical_profile_lwc.where(
    combined_output["sub_cloud_layer_mask"]
)
combined_output["vertical_profile_lwc_including_cloud"] = vertical_profile_lwc

#### Calculate skweness

In [ ]:
def calculate_skweness(data, dim="radius_bins"):
    data_m = data.mean(dim=dim)
    sk = (1 / len(data[dim]) * ((data - data_m) ** 3).sum(dim=dim)) / (
        1 / (len(data[dim]) - 1) * ((data - data_m) ** 2).sum(dim=dim) ** (3 / 2)
    )

    return sk


combined_output["mass_moment_2"] = calculate_skweness(combined_output["mass_represented"])
combined_output["mass_moment_2"].attrs.update(
    long_name="2nd Mass Moment",
    description="Mass moment 2 as the skweness of the mass distribution along radius_bins",
    units="?",
)

In [ ]:
combined_output["cloud_mass_moment_2"] = (
    combined_output["mass_moment_2"]
    .where(combined_output["gridbox"] == combined_output["max_gridbox"])
    .mean("gridbox")
)
combined_output["cloud_mass_moment_2"].attrs.update(
    long_name="Cloud 2nd Mass Moment",
    description="Mass moment 2 within the cloud",
    units="?",
)
combined_output["bottom_mass_moment_2"] = combined_output["mass_moment_2"].sel(gridbox=0)
combined_output["bottom_mass_moment_2"].attrs.update(
    long_name="Bottom 2nd Mass Moment",
    description="Mass moment 2 at the bottom of the cloud",
    units="?",
)

## Select individual clouds

In [ ]:
clouds_dict = {
    "222": dict(
        cloud_id=222,
        color="r",
        data=combined_output.sel(cloud_id=222),
    ),
    "142": dict(
        cloud_id=142,
        color="b",
        data=combined_output.sel(cloud_id=142),
    ),
}

ind_cloud_output0 = clouds_dict["222"]["data"]
ind_cloud_output1 = clouds_dict["142"]["data"]

In [ ]:
lwc = ind_cloud_output0["liquid_water_content"]
lwc_smooth = ind_cloud_output0["liquid_water_content_smooth"]
lwc_bottom_sum = lwc.sum("radius_bins", keep_attrs=True).sel(gridbox=0)
lwc_smooth_bottom_sum = lwc_smooth.sum("radius_bins", keep_attrs=True).sel(gridbox=0)

lwc_bottom_sum.attrs.update(
    long_name="Liquid Water Content",
    description="Total liquid Water Content in the surface gridbox as the sum over all radius bins.",
    units=r"$g m^{-3}$",
)
lwc_smooth_bottom_sum.attrs.update(
    long_name="Smoothed Liquid Water Content",
    description="Smoothed Total liquid Water Content in the surface gridbox as the sum over all radius bins.\nSmoothed by a rolling mean over 50 time steps.",
    units=r"$g m^{-3}$",
)
lwc_bottom_sum.name = "liquid_water_content_bottom_sum"
lwc_smooth.name = "smooth_liquid_water_content"
lwc_smooth_bottom_sum.name = "smooth_liquid_water_content_bottom_sum"


ds_select = xr.merge(
    [
        lwc,
        lwc_smooth,
        lwc_bottom_sum,
        lwc_smooth_bottom_sum,
    ]
)
ds_select.attrs = dict(
    description="Dataset with liquid water content for cloud 222",
    author="Nils Niebaum",
    date="2024/07/01",
)
ds_select["radius_bins"].attrs.update(
    long_name="Radius Bins",
    description="Radius Bins",
    units="µm",
)
ds_select["gridbox"].attrs.update(
    long_name="Gridbox",
    description="Gridbox index of rainshaft model. 0 is the bottom gridbox.",
    units="",
)
ds_select
ds_select.to_netcdf("lwc_cloud_222.nc")

## Liquid Water Content bottom gridbox / surface

In [ ]:
fig = plt.figure(figsize=(10, 4.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[0.4, 0.4, 0.03])

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
ax3 = plt.subplot(gs[2])

axs = np.array([ax1, ax2, ax3])

radius_indices = ind_cloud_output0["radius_bins"]  # .sel(radius_bins=slice(1e2, None))

resample = np.linspace(0, 1000, 10)

my_cmap = mpl.colormaps["plasma_r"].resampled(len(resample))
my_norm = mpl.colors.Normalize(vmin=0, vmax=1e3)
# colors = my_cmap(my_norm(np.round(radius_indices, decimals= -1)))
colors = my_cmap(my_norm(radius_indices))


for i, r in enumerate(radius_indices):
    axs[0].fill_between(
        ind_cloud_output0["time"],
        ind_cloud_output0["liquid_water_content_smooth"]
        .sel(gridbox=0)
        .sortby(-combined_output["radius_bins"])
        .cumsum(dim="radius_bins")
        .sel(radius_bins=r),
        color=colors[i],
    )
    axs[1].fill_between(
        ind_cloud_output1["time"],
        ind_cloud_output1["liquid_water_content_smooth"]
        .sel(gridbox=0)
        .sortby(-combined_output["radius_bins"])
        .cumsum(dim="radius_bins")
        .sel(radius_bins=r),
        color=colors[i],
    )

sc = axs[1].scatter(
    resample,
    resample,
    c=resample,
    s=0,
    norm=my_norm,
    cmap=my_cmap,
)

for ax in axs[:2]:
    ax.set_xlabel(r"Time $s$")

    set_yticks_lwc(ax)
    set_xticks_time(ax)
    ax.set_ylim(0, 0.15)
    ax.set_xlim(0, 1100)
axs[0].set_ylabel(r"Liquid water content $g m^{-3}$")
axs[1].set_yticklabels([])

fig.colorbar(
    cax=axs[2],
    mappable=sc,
    label=r"Radius $\mu m$",
    # ticks=[100, 500, 1000]
)


# fig.suptitle("Smoothed Liquid Water Content surface gridboxes")
fig.tight_layout()
fig.savefig(fig_dir / "liquid_water_content_bottom_gridboxes_radius.png", dpi=700)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

# ax.plot(
#         combined_output["time"],
#         combined_output['liquid_water_content_smooth'].sel(gridbox = 0).sum(dim="radius_bins").T,
#         color=[0.5, 0.5, 0.5],
#         linestyle="-",
#         alpha = 0.5,
#     )

ax.fill_between(
    combined_output["time"],
    combined_output["liquid_water_content_smooth"]
    .sel(gridbox=0)
    .sum(dim="radius_bins")
    .quantile(0.25, dim="cloud_id")
    .T,
    combined_output["liquid_water_content_smooth"]
    .sel(gridbox=0)
    .sum(dim="radius_bins")
    .quantile(0.75, dim="cloud_id")
    .T,
    color=[0.5, 0.5, 0.5],
    alpha=0.7,
    label="25% - 75%",
)
ax.fill_between(
    combined_output["time"],
    combined_output["liquid_water_content_smooth"]
    .sel(gridbox=0)
    .sum(dim="radius_bins")
    .quantile(0.1, dim="cloud_id")
    .T,
    combined_output["liquid_water_content_smooth"]
    .sel(gridbox=0)
    .sum(dim="radius_bins")
    .quantile(0.9, dim="cloud_id")
    .T,
    color=[0.5, 0.5, 0.5],
    alpha=0.2,
    label="10% - 90%",
)
ax.plot(
    combined_output["time"],
    combined_output["liquid_water_content_smooth"]
    .sel(gridbox=0)
    .sum(dim="radius_bins")
    .median("cloud_id")
    .T,
    color="k",
    linestyle="-",
    alpha=0.5,
    label="median",
)

ax.plot(
    ind_cloud_output0["time"],
    ind_cloud_output0["liquid_water_content_smooth"].sel(gridbox=0).sum(dim="radius_bins").T,
    color="r",
    linestyle="-",
    alpha=0.5,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)

ax.plot(
    ind_cloud_output1["time"],
    ind_cloud_output1["liquid_water_content_smooth"].sel(gridbox=0).sum(dim="radius_bins").T,
    color="b",
    linestyle="-",
    alpha=0.5,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


ax.set_ylabel(r"Liquid water content $g m^{-3}$")
ax.set_xlabel(r"Time $s$")


set_yticks_lwc(ax)
set_xticks_time(ax)

ax.set_ylim(0, 0.15)
ax.set_xlim(0, None)

# ax.set_title("Liquid Water Content in surface gridbox")
ax.legend(loc="lower right")

fig.savefig(fig_dir / "liquid_water_content_bottom_gridboxes_all_full.png", dpi=700)
fig.savefig(fig_dir / "liquid_water_content_bottom_gridboxes_all_full.svg")

<module 'numpy' from '/work/um1487/m301096/conda/envs/sdm_eurec4a_env312/lib/python3.12/site-packages/numpy/__init__.py'>

## Vertical profiles

#### Liquid water content along the vertical

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

ax.plot(
    combined_output["vertical_profile_lwc"].T,
    combined_output["pseudo_coord3"],
    c=[0.5, 0.5, 0.5],
    alpha=0.2,
)

ax.plot(
    ind_cloud_output0["vertical_profile_lwc"],
    ind_cloud_output0["pseudo_coord3"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    ind_cloud_output1["vertical_profile_lwc"],
    ind_cloud_output1["pseudo_coord3"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_lwc"].long_name
    + f" {combined_output['vertical_profile_lwc'].attrs['units']}"
)
ax.set_xlabel(xlabel)

set_yticks_height(ax)
ax.set_ylim(0, 1200)

ax.set_ylabel("Height $m$")
ax.set_xticks([0, 0.05, 0.1, 0.15])
ax.set_xlim(0, 0.16)
ax.legend(loc="upper left")

# ax.set_title("Liquid water content")

fig.savefig(fig_dir / "vertical_LWC_all.png", dpi=700)
fig.savefig(fig_dir / "vertical_LWC_all.svg")

In [ ]:
plt.plot(combined_output["cloud_id"].astype(str), combined_output["vertical_profile_lwc"].max("gridbox"))
plt.plot(combined_output["cloud_id"].astype(str), combined_output["vertical_profile_lwc"].sel(gridbox=0))
plt.tick_params(axis="x", rotation=-90)
plt.tight_layout()

#### LWC fraction compared to top layer $\%$  

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

ax.plot(
    combined_output["vertical_profile_mass_fraction"].T,
    combined_output["pseudo_coord3"],
    c=[0.5, 0.5, 0.5],
    alpha=0.2,
)

ax.plot(
    ind_cloud_output0["vertical_profile_mass_fraction"],
    ind_cloud_output0["pseudo_coord3"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    ind_cloud_output1["vertical_profile_mass_fraction"],
    ind_cloud_output1["pseudo_coord3"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_mass_fraction"].long_name
    + f" {combined_output['vertical_profile_mass_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 1200)

ax.set_ylabel("Height $m$")
ax.set_xticks([60, 80, 100])
ax.set_xlim(60, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

# ax.set_title("Mass fraction relative to top sub-cloud layer")

fig.savefig(fig_dir / "vertical_mass_fraction_all.png", dpi=700)
fig.savefig(fig_dir / "vertical_mass_fraction_all.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

# ax.plot(
#     combined_output["vertical_profile_mass_fraction"].T,
#     combined_output["pseudo_coord3"],
#     c=[0.5, 0.5, 0.5],
#     alpha=0.2,
# )

ax.plot(
    ind_cloud_output0["vertical_profile_mass_fraction"],
    ind_cloud_output0["pseudo_coord3"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    ind_cloud_output1["vertical_profile_mass_fraction"],
    ind_cloud_output1["pseudo_coord3"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_mass_fraction"].long_name
    + f" {combined_output['vertical_profile_mass_fraction'].attrs['units']}"
)
ax.set_xlabel(xlabel)
# ax.set_title(ind_cloud_output1[vertical_profile_mass_fraction].long_name)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks([60, 80, 100])
ax.set_xlim(60, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

# ax.set_title("Mass fraction relative to top sub-cloud layer")

fig.savefig(fig_dir / "vertical_mass_fraction_individual.png", dpi=700)
fig.savefig(fig_dir / "vertical_mass_fraction_individual.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

ax.plot(
    combined_output["vertical_profile_mass_fraction"].T,
    combined_output["pseudo_coord3_normalized"],
    c=[0.5, 0.5, 0.5],
    alpha=0.2,
)

ax.plot(
    ind_cloud_output0["vertical_profile_mass_fraction"],
    combined_output["pseudo_coord3_normalized"].sel(cloud_id=ind_cloud_output0["cloud_id"]),
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    ind_cloud_output1["vertical_profile_mass_fraction"],
    combined_output["pseudo_coord3_normalized"].sel(cloud_id=ind_cloud_output1["cloud_id"]),
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_mass_fraction"].long_name
    + f" {combined_output['vertical_profile_mass_fraction'].units}"
)
ax.set_xlabel(xlabel)

ax.set_yticks([0, 1])
ax.set_ylim(0, 1)
ax.set_ylabel("Normalized height")
ax.set_xticks([60, 80, 100])
ax.set_xlim(60, 105)
ax.axvline(100, color="k", linestyle="--", alpha=0.5)
ax.legend(loc="upper left")

# ax.set_title("Mass fraction relative to top sub-cloud layer")
fig.savefig(fig_dir / "vertical_mass_fraction_all_normalized_height.png", dpi=700)
fig.savefig(fig_dir / "vertical_mass_fraction_all_normalized_height.svg")

#### Liquid Water Content MINUS Top Liquid water content $gm^{-3}$

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

ax.plot(
    (
        combined_output["vertical_profile_lwc"]
        - combined_output["vertical_profile_lwc"].sel(gridbox=combined_output["max_gridbox"] - 1)
    ).T,
    combined_output["pseudo_coord3"],
    c=[0.5, 0.5, 0.5],
    alpha=0.2,
)

ax.plot(
    (
        ind_cloud_output0["vertical_profile_lwc"]
        - ind_cloud_output0["vertical_profile_lwc"].sel(gridbox=ind_cloud_output0["max_gridbox"] - 1)
    ),
    ind_cloud_output0["pseudo_coord3"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    (
        ind_cloud_output1["vertical_profile_lwc"]
        - ind_cloud_output1["vertical_profile_lwc"].sel(gridbox=ind_cloud_output1["max_gridbox"] - 1)
    ),
    ind_cloud_output1["pseudo_coord3"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_lwc"].long_name
    + f" {combined_output['vertical_profile_lwc'].attrs['units']}"
)
ax.set_xlabel(xlabel)

set_yticks_height(ax)
ax.set_ylim(0, 1200)

ax.set_ylabel("Height $m$")
ax.set_xticks([-0.01, 0])
ax.set_xlim(-0.015, 0.005)
ax.legend(loc="upper left")

# ax.set_title("Liquid water content")

fig.savefig(fig_dir / "vertical_LWC_all_minus_top.png", dpi=700)
fig.savefig(fig_dir / "vertical_LWC_all_minus_top.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

# ax.plot(
#     (combined_output["vertical_profile_lwc"] - combined_output["vertical_profile_lwc"].sel(gridbox = combined_output["max_gridbox"] - 1)).T,
#     combined_output["pseudo_coord3"],
#     c=[0.5, 0.5, 0.5],
#     alpha=0.2,
# )

ax.plot(
    (
        ind_cloud_output0["vertical_profile_lwc"]
        - ind_cloud_output0["vertical_profile_lwc"].sel(gridbox=ind_cloud_output0["max_gridbox"] - 1)
    ),
    ind_cloud_output0["pseudo_coord3"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    (
        ind_cloud_output1["vertical_profile_lwc"]
        - ind_cloud_output1["vertical_profile_lwc"].sel(gridbox=ind_cloud_output1["max_gridbox"] - 1)
    ),
    ind_cloud_output1["pseudo_coord3"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_lwc"].long_name
    + f" {combined_output['vertical_profile_lwc'].attrs['units']}"
)
ax.set_xlabel(xlabel)

set_yticks_height(ax)
ax.set_ylim(0, 800)

ax.set_ylabel("Height $m$")
ax.set_xticks([-0.01, 0])
ax.set_xlim(-0.015, 0.005)
ax.legend(loc="upper left")

# ax.set_title("Liquid water content")

fig.savefig(fig_dir / "vertical_LWC_individual_minus_top.png", dpi=700)
fig.savefig(fig_dir / "vertical_LWC_individual_minus_top.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

ax.plot(
    (
        combined_output["vertical_profile_lwc"]
        - combined_output["vertical_profile_lwc"].sel(gridbox=combined_output["max_gridbox"] - 1)
    ).T,
    combined_output["pseudo_coord3_normalized"],
    c=[0.5, 0.5, 0.5],
    alpha=0.2,
)

ax.plot(
    (
        ind_cloud_output0["vertical_profile_lwc"]
        - ind_cloud_output0["vertical_profile_lwc"].sel(gridbox=ind_cloud_output0["max_gridbox"] - 1)
    ),
    ind_cloud_output0["pseudo_coord3_normalized"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    (
        ind_cloud_output1["vertical_profile_lwc"]
        - ind_cloud_output1["vertical_profile_lwc"].sel(gridbox=ind_cloud_output1["max_gridbox"] - 1)
    ),
    ind_cloud_output1["pseudo_coord3_normalized"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_lwc"].long_name
    + f" {combined_output['vertical_profile_lwc'].attrs['units']}"
)
ax.set_xlabel(xlabel)

# set_yticks_height(ax)
ax.set_yticks([0, 1])
ax.set_ylim(0, 1.1)

ax.set_ylabel("Height $m$")
ax.set_xticks([-0.01, 0])
ax.set_xlim(-0.015, 0.005)
ax.legend(loc="upper left")

fig.savefig(fig_dir / "vertical_LWC_all_minus_top_norm.png", dpi=700)
fig.savefig(fig_dir / "vertical_LWC_all_minus_top_norm.svg")

#### Evaporation rates / Mass difference $g m^{-3} s^{-1}$

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))

ax.plot(
    1e3 * combined_output["vertical_profile_mass_diff"].T,
    combined_output["pseudo_coord3_normalized"],
    c=[0.5, 0.5, 0.5],
    alpha=0.2,
)

ax.plot(
    1e3 * ind_cloud_output0["vertical_profile_mass_diff"],
    ind_cloud_output0["pseudo_coord3_normalized"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    1e3 * ind_cloud_output1["vertical_profile_mass_diff"],
    ind_cloud_output1["pseudo_coord3_normalized"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


xlabel = (
    combined_output["vertical_profile_mass_diff"].long_name
    + f" {combined_output['vertical_profile_mass_diff'].attrs['units']}"
)
ax.set_xlabel(xlabel)

# set_yticks_height(ax)
ax.set_yticks([0, 1])
ax.set_ylim(0, 1.1)

ax.set_ylabel("Height $m$")
# ax.set_xticks([-0.01, 0])
# ax.set_xlim(-0.015, 0.005)
ax.legend(loc="upper left")

# ax.set_title("Liquid water content")

fig.savefig(fig_dir / "vertical_profile_mass_diff.png", dpi=700)
fig.savefig(fig_dir / "vertical_profile_mass_diff.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))


sc = ax.scatter(
    1e3 * combined_output["vertical_profile_mass_diff"].T,
    combined_output["pseudo_coord3_normalized"],
    c=combined_output["vertical_profile_mass_fraction"].T,
    alpha=0.7,
    marker=".",
    cmap="inferno",
)
ax.plot(
    1e3 * ind_cloud_output0["vertical_profile_mass_diff"],
    ind_cloud_output0["pseudo_coord3_normalized"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    1e3 * ind_cloud_output1["vertical_profile_mass_diff"],
    ind_cloud_output1["pseudo_coord3_normalized"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


fig.colorbar(ax=ax, mappable=sc, label=r"Mass Fraction $\%$")


xlabel = (
    combined_output["vertical_profile_mass_diff"].long_name
    + f" {combined_output['vertical_profile_mass_diff'].attrs['units']}"
)
ax.set_xlabel(xlabel)

# set_yticks_height(ax)
ax.set_yticks([0, 1])
ax.set_ylim(0, 1.1)

ax.set_ylabel("Height $m$")
ax.set_xticks([-0.05, 0])
ax.set_xlim(-0.06, 0.0)
ax.legend(loc="upper left")

# ax.set_title("Liquid water content")

fig.savefig(fig_dir / "vertical_profile_mass_diff_COLOR.png", dpi=700)
fig.savefig(fig_dir / "vertical_profile_mass_diff_COLOR.svg")

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 4.5))


ax.plot(
    1e3 * combined_output["vertical_profile_mass_diff"].T,
    combined_output["pseudo_coord3_normalized"],
    alpha=0.2,
    c=[0.5, 0.5, 0.5],
)
sc = ax.scatter(
    1e3 * combined_output["vertical_profile_mass_diff"].T,
    combined_output["pseudo_coord3_normalized"],
    c=combined_output["vertical_profile_mass_fraction"].T,
    alpha=0.7,
    marker=".",
    cmap="inferno",
)
ax.plot(
    1e3 * ind_cloud_output0["vertical_profile_mass_diff"],
    ind_cloud_output0["pseudo_coord3_normalized"],
    c="r",
    alpha=1,
    label=f"cloud {ind_cloud_output0['cloud_id'].values}",
)
ax.plot(
    1e3 * ind_cloud_output1["vertical_profile_mass_diff"],
    ind_cloud_output1["pseudo_coord3_normalized"],
    c="b",
    alpha=1,
    label=f"cloud {ind_cloud_output1['cloud_id'].values}",
)


fig.colorbar(ax=ax, mappable=sc, label=r"Mass Fraction $\%$")


xlabel = (
    combined_output["vertical_profile_mass_diff"].long_name
    + f" {combined_output['vertical_profile_mass_diff'].attrs['units']}"
)
ax.set_xlabel(xlabel)

# set_yticks_height(ax)
ax.set_yticks([0, 1])
ax.set_ylim(0, 1.1)

ax.set_ylabel("Height $m$")
ax.set_xticks([-0.05, 0])
ax.set_xlim(-0.06, 0.0)
ax.legend(loc="upper left")

# ax.set_title("Liquid water content")

fig.savefig(fig_dir / "vertical_profile_mass_diff_COLOR_LINE.png", dpi=700)
fig.savefig(fig_dir / "vertical_profile_mass_diff_COLOR_LINE.svg")

### Mass change depending on radius

In [ ]:
def pcolormesh_evaporation(data):
    data = (
        data.sel(time=slice(400, None))
        .sel(radius_bins=slice(0, None))
        .mean(dim="time")
        .where(data["gridbox"] < data["max_gridbox"])
    )

    r_index = data["radius_bins"]
    g_index = ind_cloud_output1["pseudo_coord3"]

    rr, gg = np.meshgrid(r_index, g_index)

    my_cmap = mpl.colormaps["inferno_r"]
    my_norm = mpl.colors.LogNorm(
        vmin=data["mass_difference"].min().values,
        vmax=data["mass_difference"].max().values,
    )

    plot_data = {
        "mmc": dict(
            data=data["mass_difference"] * 1e3,
            cmap="PuOr",
            clabel=r"($\overline{dm}$)  $gs^{-1}$",
            title=r"Mean (temporal) mass change ($\overline{dm}$)",
            vmin=1e0,
            vmax=-1e0,
            norm_func=mpl.colors.SymLogNorm,
            norm_args=dict(linthresh=1e-3),
        ),
        "mmc_normalized": dict(
            data=(data["mass_difference"] - data["mass_difference"].mean(dim="gridbox"))
            / data["mass_difference"].std(dim="gridbox"),
            cmap="PuOr",
            clabel=r"$\overline{dm}_{norm}$",
            title=r"$\overline{dm}$ normalized for each radius bin",
            vmin=2,
            vmax=-2,
            norm_func=mpl.colors.Normalize,
            norm_args=dict(),
        ),
    }

    fig, axss = plt.subplots(ncols=2, nrows=1, sharex=True, figsize=(10, 4.5))

    axs = axss.flatten()

    for ax, setup_name in zip(axs.flatten(), plot_data):
        setup = plot_data[setup_name]

        da = setup["data"]
        if setup["vmin"] == None:
            vmin = da.min().values
        else:
            vmin = setup["vmin"]

        if setup["vmax"] == None:
            vmax = da.max().values
        else:
            vmax = setup["vmax"]

        my_cmap = mpl.colormaps[setup["cmap"]]
        my_norm = setup["norm_func"](vmin=vmin, vmax=vmax, **setup["norm_args"])

        sc = ax.pcolormesh(
            rr,
            gg,
            da.transpose("gridbox", "radius_bins"),
            norm=my_norm,
            cmap=my_cmap,
            shading="nearest",
        )
        fig.colorbar(ax=ax, mappable=sc, label=setup["clabel"])
        ax.set_title(setup["title"])

    indices = combined_output["time"].sel(time=slice(1, None))

    my_cmap = mpl.colormaps["plasma_r"]
    my_norm = mpl.colors.Normalize(vmin=indices.min(), vmax=indices.max())
    colors = my_cmap(my_norm(indices))

    axs[0].set_xscale("log")
    for ax in axs.flatten():
        ax.set_xlabel(r"Radius $µm$")
        ax.set_ylabel(r"Height $m$")
        set_logxticks_micrometer(ax)
        set_yticks_height(ax)
        ax.set_ylim(0, 700)
        ax.set_xlim(1e-1, 4e3)

    # fig.suptitle(f"Mass change radius dependency cloud {data['cloud_id'].values}")

    fig.tight_layout()
    return fig, axs


fig, axs = pcolormesh_evaporation(ind_cloud_output0)
fig.savefig(
    fig_dir / f"mass_change_radius_dependency_{ind_cloud_output0['cloud_id'].values}.png", dpi=700
)
fig, axs = pcolormesh_evaporation(ind_cloud_output1)
fig.savefig(
    fig_dir / f"mass_change_radius_dependency_{ind_cloud_output1['cloud_id'].values}.png", dpi=700
)

In [ ]:
def plot_evaporation_radius_scatter(data):
    r_index = data["radius_bins"]
    rr = data["mass_represented"] * 0 + r_index
    g_index = data["gridbox"]
    gg = data["mass_represented"] * 0 + g_index

    fig, axs = plt.subplots(ncols=2, sharey=True, figsize=(10, 3.5))

    my_cmap = mpl.colormaps["plasma_r"]
    my_norm = mpl.colors.LogNorm(vmin=r_index.min().values, vmax=r_index.max().values)
    my_cmap = mpl.colors.ListedColormap(my_cmap(my_norm(r_index[::2])), name="myColorMap")

    sc = axs[0].scatter(
        x=data["mass_represented"] * 1e3,
        y=data["mass_difference"] * 1e3,
        c=rr,
        # s = ss,
        norm=my_norm,
        cmap=my_cmap,
        marker=".",
    )
    fig.colorbar(ax=axs[0], mappable=sc, label="radius in µm")

    my_cmap = mpl.colormaps["cividis_r"]
    my_norm = mpl.colors.Normalize(vmin=g_index.min().values, vmax=g_index.max().values)
    my_cmap = mpl.colors.ListedColormap(my_cmap(my_norm(g_index[::2])), name="myColorMap")

    sc = axs[1].scatter(
        x=rr,
        y=data["mass_difference"] * 1e3,
        c=gg,
        # s = ss,
        norm=my_norm,
        cmap=my_cmap,
        marker=".",
    )
    fig.colorbar(ax=axs[1], mappable=sc, label="gridbox (0 bottom)")

    # fig.suptitle("Dependency of evaporation on radius and gridbox.")
    axs[0].set_title("colored by radius")
    axs[1].set_title("colored by gridbox")

    axs[0].set_ylabel(r"Mean mass change ($\overline{dm}$) [g/s]")
    axs[0].set_xlabel(r"Mass ($m$) [g]")
    axs[1].set_ylabel(r"Mean mass change ($\overline{dm}$) [g/s]")
    axs[1].set_xlabel("Radius [µm]")
    axs[1].set_xscale("log")
    fig.tight_layout()
    return fig, axs


data = ind_cloud_output0.sel(time=slice(400, None)).sel(radius_bins=slice(1e1, None)).mean(dim="time")
fig, axs = plot_evaporation_radius_scatter(data)
fig.savefig(
    fig_dir / f"evaporation_radius_dependency_scatter_cloud{data['cloud_id'].values}.png", dpi=700
)

data = ind_cloud_output1.sel(time=slice(400, None)).sel(radius_bins=slice(1e1, None)).mean(dim="time")
fig, axs = plot_evaporation_radius_scatter(data)
fig.savefig(
    fig_dir / f"evaporation_radius_dependency_scatter_cloud{data['cloud_id'].values}.png", dpi=700
)

### Multiplicity and PSD for example clouds 

In [ ]:
def select_gridboxes(data):
    return data["gridbox"].sel(
        gridbox=[data["max_gridbox"], data["max_gridbox"] - 1, 25, 15, 5, 0], method="nearest"
    )

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4.5))

xi_mean0 = ind_cloud_output0["xi"].sel(time=slice(400, None)).mean(dim="time")
xi_mean0 = xi_mean0.where(xi_mean0 > 0)
norm_alt0 = ind_cloud_output0["pseudo_coord3_full"]

xi_mean1 = ind_cloud_output1["xi"].sel(time=slice(400, None)).mean(dim="time")
xi_mean1 = xi_mean1.where(xi_mean1 > 0)
norm_alt1 = ind_cloud_output1["pseudo_coord3_full"]

selected_gridboxes0 = select_gridboxes(ind_cloud_output0)
selected_gridboxes1 = select_gridboxes(ind_cloud_output1)

colors = mpl.colormaps.get_cmap("cividis")(np.linspace(0, 1, len(selected_gridboxes0)))

for i, g in enumerate(selected_gridboxes0):
    alt0 = norm_alt0.sel(gridbox=g)

    axs[0].scatter(
        xi_mean0["radius_bins"],
        xi_mean0.sel(gridbox=g),
        label=f"{alt0.values} m",
        color=colors[i],
        alpha=1,
        marker=".",
    )
    axs[0].set_title(f"Cloud {ind_cloud_output0['cloud_id'].values}")

for i, g in enumerate(selected_gridboxes1):
    alt1 = norm_alt1.sel(gridbox=g)
    axs[1].scatter(
        xi_mean1["radius_bins"],
        xi_mean1.sel(gridbox=g),
        label=f"{alt1.values} m",
        color=colors[i],
        alpha=1,
        marker=".",
    )
    axs[1].set_title(f"Cloud {ind_cloud_output1['cloud_id'].values}")

for ax in axs:
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.legend(loc="lower left", ncol=3, handleheight=1, handlelength=0.05, labelspacing=0.005)
    set_logxticks_micrometer(ax)
    set_logtyticks_psd(ax)
    ax.set_xlim(1e-1, 4e3)
    ax.set_ylim(1e0, 1e9)
    ax.set_xlabel(r"Radius $µm$")

axs[0].set_ylabel(r"Multiplicity ($\xi$)")

# fig.suptitle("Droplet distribution at different heights")
fig.tight_layout()
fig.savefig(fig_dir / "xi_distribution_height.png", dpi=700)
fig.savefig(fig_dir / "xi_distribution_height.svg")

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4.5))

xi_mean0 = ind_cloud_output0["xi"].sel(time=slice(400, None)).mean(dim="time")
xi_mean0 = xi_mean0.where(xi_mean0 > 0) / ind_cloud_output0["pseudo_gribox_volume_full"]
norm_alt0 = ind_cloud_output0["pseudo_coord3_full"]

xi_mean1 = ind_cloud_output1["xi"].sel(time=slice(400, None)).mean(dim="time")
xi_mean1 = xi_mean1.where(xi_mean1 > 0) / ind_cloud_output1["pseudo_gribox_volume_full"]
norm_alt1 = ind_cloud_output1["pseudo_coord3_full"]

selected_gridboxes0 = select_gridboxes(ind_cloud_output0)
selected_gridboxes1 = select_gridboxes(ind_cloud_output1)

colors = mpl.colormaps.get_cmap("cividis")(np.linspace(0, 1, len(selected_gridboxes0)))

for i, g in enumerate(selected_gridboxes0):
    alt0 = norm_alt0.sel(gridbox=g)

    axs[0].scatter(
        xi_mean0["radius_bins"],
        xi_mean0.sel(gridbox=g),
        label=f"{alt0.values} m",
        color=colors[i],
        alpha=1,
        marker=".",
    )
    axs[0].set_title(f"Cloud {ind_cloud_output0['cloud_id'].values}")

for i, g in enumerate(selected_gridboxes1):
    alt1 = norm_alt1.sel(gridbox=g)
    axs[1].scatter(
        xi_mean1["radius_bins"],
        xi_mean1.sel(gridbox=g),
        label=f"{alt1.values} m",
        color=colors[i],
        alpha=1,
        marker=".",
    )
    axs[1].set_title(f"Cloud {ind_cloud_output1['cloud_id'].values}")

for ax in axs:
    ax.set_yscale("log")
    ax.set_xscale("log")
    ax.legend(loc="upper left", ncol=3, handleheight=1, handlelength=0.05, labelspacing=0.005)
    set_logxticks_micrometer(ax)
    set_logtyticks_psd(ax)
    ax.set_xlim(1e-1, 4e3)
    ax.set_ylim(1e-2, 1e6)
    ax.set_xlabel(r"Radius $µm$")

axs[0].set_ylabel(r"Droplet concentration $\# m^{-3}$")

# fig.suptitle("Droplet distribution at different heights")
fig.tight_layout()
fig.savefig(fig_dir / "psd_distribution_height.png", dpi=700)
fig.savefig(fig_dir / "psd_distribution_height.svg")

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4.5))

xi_mean0 = ind_cloud_output0["number_superdroplets"].sel(time=slice(400, None)).mean(dim="time")
xi_mean0 = xi_mean0.where(xi_mean0 > 0)  # / ind_cloud_output0["pseudo_gribox_volume_full"]
norm_alt0 = ind_cloud_output0["pseudo_coord3_full"]

xi_mean1 = ind_cloud_output1["number_superdroplets"].sel(time=slice(400, None)).mean(dim="time")
xi_mean1 = xi_mean1.where(xi_mean1 > 0)  # / ind_cloud_output1["pseudo_gribox_volume_full"]
norm_alt1 = ind_cloud_output1["pseudo_coord3_full"]

selected_gridboxes0 = select_gridboxes(ind_cloud_output0)
selected_gridboxes1 = select_gridboxes(ind_cloud_output1)

colors = mpl.colormaps.get_cmap("cividis")(np.linspace(0, 1, len(selected_gridboxes0)))

for i, g in enumerate(selected_gridboxes0):
    alt0 = norm_alt0.sel(gridbox=g)

    axs[0].scatter(
        xi_mean0["radius_bins"],
        xi_mean0.sel(gridbox=g),
        label=f"{alt0.values} m",
        color=colors[i],
        alpha=1,
        marker=".",
    )
    axs[0].set_title(f"Cloud {ind_cloud_output0['cloud_id'].values}")

for i, g in enumerate(selected_gridboxes1):
    alt1 = norm_alt1.sel(gridbox=g)
    axs[1].scatter(
        xi_mean1["radius_bins"],
        xi_mean1.sel(gridbox=g),
        label=f"{alt1.values} m",
        color=colors[i],
        alpha=1,
        marker=".",
    )
    axs[1].set_title(f"Cloud {ind_cloud_output1['cloud_id'].values}")

for ax in axs:
    # ax.set_yscale("log")
    ax.set_xscale("log")
    ax.legend(loc="upper left", ncol=3, handleheight=1, handlelength=0.05, labelspacing=0.005)
    set_logxticks_micrometer(ax)
    ax.set_yticks([0, 50, 100])
    ax.set_ylim(0, 100)
    ax.set_xlim(1e-1, 4e3)
    # ax.set_ylim(1e-2, 1e6)
    ax.set_xlabel(r"Radius $µm$")

axs[0].set_ylabel(r"Number Superdroplets $\# m^{-3}$")

# fig.suptitle("Droplet distribution at different heights")
fig.tight_layout()
fig.savefig(fig_dir / "number_sds_height.png", dpi=700)
fig.savefig(fig_dir / "number_sds_height.svg")

## All clouds plots 

In [ ]:
def errorbar_and_color(
    ax, x: xr.DataArray, y: xr.DataArray, xerr: xr.DataArray, yerr: xr.DataArray, c="k", draw_line=True
) -> mpl.collections.PathCollection:
    sc = ax.errorbar(x, y, xerr=xerr, yerr=yerr, c=c, alpha=0.1, linestyle="None")
    ax.set_xscale("log")
    ax.set_yscale("log")

    ax.set_xlabel(x.long_name + f" {x.units}")
    ax.set_ylabel(y.long_name + f" {y.units}")
    return sc


def scatter_and_color(
    ax,
    x: xr.DataArray,
    y: xr.DataArray,
    c=xr.DataArray,
    draw_line=True,
    kwargs=dict(alpha=0.7, linestyle="None"),
) -> mpl.collections.PathCollection:
    sc = ax.scatter(x, y, c=c, **kwargs)
    ax.set_xscale("log")
    ax.set_yscale("log")

    ax.set_xlabel(x.long_name + f" {x.units}")
    ax.set_ylabel(y.long_name + f" {y.units}")
    return sc


fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(10, 4.5), sharey=False)

sc = scatter_and_color(
    axs[0],
    combined_output["top_subcloud_mass"],
    combined_output["bottom_subcloud_mass"],
    combined_output["mass_fraction_total"],
)

fig.colorbar(sc, ax=axs[0], label="Mass Fraction [%]")

sc = axs[0].scatter(
    ind_cloud_output0["top_subcloud_mass"],
    ind_cloud_output0["bottom_subcloud_mass"],
    c="r",
    s=100,
    marker="x",
)
sc = axs[0].scatter(
    ind_cloud_output1["top_subcloud_mass"],
    ind_cloud_output1["bottom_subcloud_mass"],
    c="b",
    s=100,
    marker="x",
)

sc = scatter_and_color(
    axs[1],
    combined_output["cloud_mass_moment_2"].sel(time=slice(400, None)).mean("time", keep_attrs=True),
    combined_output["bottom_mass_moment_2"].sel(time=slice(400, None)).mean("time", keep_attrs=True),
    combined_output["mass_fraction_total"],
)
fig.colorbar(sc, ax=axs[1], label="Mass Fraction [%]")

sc = axs[1].scatter(
    ind_cloud_output0["cloud_mass_moment_2"].sel(time=slice(400, None)).mean("time", keep_attrs=True),
    ind_cloud_output0["bottom_mass_moment_2"].sel(time=slice(400, None)).mean("time", keep_attrs=True),
    c="r",
    s=100,
    marker="x",
)
sc = axs[1].scatter(
    ind_cloud_output1["cloud_mass_moment_2"].sel(time=slice(400, None)).mean("time", keep_attrs=True),
    ind_cloud_output1["bottom_mass_moment_2"].sel(time=slice(400, None)).mean("time", keep_attrs=True),
    c="b",
    s=100,
    marker="x",
)


axs[1].set_xscale("linear")
axs[1].set_yscale("linear")

axs[0].set_title("Mass change by surface mass fraction")
axs[1].set_title("2nd mass moment by surface mass fraction")

axs[1].set_xticks([0.3, 0.6])
axs[1].set_xlim(0.3, 0.8)
axs[1].set_yticks([0.3, 0.6])
axs[1].set_ylim(0.3, 0.8)


fig.tight_layout()

fig.savefig(fig_dir / "mass_change_surface_mass_fraction.png", dpi=700)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4.5))

sc = errorbar_and_color(
    ax,
    x=combined_output["cloud_mass_moment_2"].sel(time=slice(400, None)).median("time", keep_attrs=True),
    xerr=np.abs(
        combined_output["cloud_mass_moment_2"]
        .sel(time=slice(400, None))
        .quantile([0.33, 0.66], dim="time")
        - combined_output["cloud_mass_moment_2"]
        .sel(time=slice(400, None))
        .median("time", keep_attrs=True)
    ),
    y=combined_output["bottom_mass_moment_2"].sel(time=slice(400, None)).median("time", keep_attrs=True),
    yerr=np.abs(
        combined_output["bottom_mass_moment_2"]
        .sel(time=slice(400, None))
        .quantile([0.33, 0.66], dim="time")
        - combined_output["bottom_mass_moment_2"]
        .sel(time=slice(400, None))
        .median("time", keep_attrs=True)
    ),
    c="k",
    draw_line=False,
)
sc = scatter_and_color(
    ax,
    x=combined_output["cloud_mass_moment_2"].sel(time=slice(400, None)).median("time", keep_attrs=True),
    y=combined_output["bottom_mass_moment_2"].sel(time=slice(400, None)).median("time", keep_attrs=True),
    c=combined_output["mass_fraction_total"],
)

ax.set_xscale("linear")
ax.set_yscale("linear")

In [ ]:
fig, ax = plt.subplots(figsize=(4.5, 4.5))

sc = errorbar_and_color(
    ax,
    x=combined_output["cloud_mass_moment_2"].sel(time=slice(400, None)).median("time", keep_attrs=True),
    xerr=np.abs(
        combined_output["cloud_mass_moment_2"]
        .sel(time=slice(400, None))
        .quantile([0.33, 0.66], dim="time")
        - combined_output["cloud_mass_moment_2"]
        .sel(time=slice(400, None))
        .median("time", keep_attrs=True)
    ),
    y=combined_output["liquid_water_content"]
    .sel(gridbox=0)
    .sum(dim="radius_bins", keep_attrs=True)
    .sel(time=slice(400, None))
    .median("time", keep_attrs=True),
    yerr=np.abs(
        combined_output["liquid_water_content"]
        .sel(gridbox=0)
        .sum(dim="radius_bins", keep_attrs=True)
        .sel(time=slice(400, None))
        .quantile([0.33, 0.66], dim="time")
        - combined_output["liquid_water_content"]
        .sel(gridbox=0)
        .sum(dim="radius_bins")
        .sel(time=slice(400, None))
        .median("time", keep_attrs=True)
    ),
    c="k",
    draw_line=False,
)
sc = scatter_and_color(
    ax,
    x=combined_output["cloud_mass_moment_2"].sel(time=slice(400, None)).median("time", keep_attrs=True),
    y=combined_output["liquid_water_content"]
    .sel(gridbox=0)
    .sum(dim="radius_bins", keep_attrs=True)
    .sel(time=slice(400, None))
    .median("time", keep_attrs=True),
    c=combined_output["mass_fraction_total"],
)

ax.set_xscale("log")
ax.set_yscale("log")